In [68]:
# Importing libraries
import numpy as np
from numpy import kron
from scipy import identity
from scipy.sparse.linalg import eigsh
import scipy.linalg.lapack as la
import math
from collections import namedtuple 

In [69]:
# Step1: constructing one-qubit operator
splus=np.zeros([2,2], dtype='float64')
sminus=np.zeros([2,2],dtype='float64')
sz=np.zeros([2,2],dtype='float64')
splus[0,1]=1
sminus[1,0]=1
sz[0,0]=0.5
sz[1,1]=-0.5
ide=np.eye(2,dtype='float64')

In [70]:
# Step2: Constructing two-qubit Hamiltonian
H2=kron(sz,sz)+0.5*(kron(splus,sminus)+kron(sminus,splus))

In [71]:
Block = namedtuple("Block", ["number_spins", "basis_size", "operator_dict"])
increment_block=namedtuple("increment_block", ["number_spins", "basis_size", "operator_dict"])
intial_block=Block(number_spins=2,basis_size=4,\
                   operator_dict={'ham':H2,'sz':kron(ide,sz),'sp':kron(ide,splus),'sm':kron(ide,sminus),\
                                  'id':kron(ide,ide)})


In [72]:
def block_increment(input_block):
    HL_new=kron(input_block.operator_dict['ham'],ide)\
    +kron(input_block.operator_dict['sz'],sz)+\
    0.5*(kron(input_block.operator_dict['sp'],sminus)+kron(input_block.operator_dict['sm'],splus))
    enlarged_operator_dictionary={'ham':HL_new,'sz':kron(input_block.operator_dict['id'],sz),\
                                 'sp':kron(input_block.operator_dict['id'],splus),\
                                 'sm':kron(input_block.operator_dict['id'],sminus),\
                                 'id':kron(input_block.operator_dict['id'],ide)}
    output_block=increment_block(number_spins=input_block.number_spins+1,basis_size=input_block.basis_size*2,\
                                operator_dict=enlarged_operator_dictionary)
    return(output_block)

In [85]:
def dmrg_step(sysblock,envblock,m):
    sysblock_new=block_increment(sysblock)
    if sysblock is envblock:  # no need to recalculate a second time
        envblock_new = sysblock_new
    else:
        envblock_new = envblock
    new_block=sysblock_new
    sys_enl_op = sysblock_new.operator_dict
    env_enl_op = envblock_new.operator_dict
    if sysblock_new.basis_size > m:
        super_ham=kron(sys_enl_op['ham'],env_enl_op['id'])+\
        kron(sys_enl_op['id'],env_enl_op ['ham'])+\
        kron(sys_enl_op['sz'],env_enl_op ['sz'])+\
        0.5*(kron(sys_enl_op['sp'],env_enl_op ['sm'])+\
             kron(sys_enl_op['sm'],env_enl_op ['sp']))
        eigval, eigvec = eigsh(super_ham, k=1,which='SA')
        print(eigval)
        eigvec=eigvec.reshape(sysblock_new.basis_size,envblock_new.basis_size)
        rho_red=np.matmul(eigvec,np.transpose(eigvec))
        eval_red,evec_red,info=la.dsyev(rho_red)
        tf_matrix=evec_red[:,np.shape(rho_red)[0]-m:np.shape(rho_red)[0]]
        new_dictionary={'ham':np.matmul(np.matmul(np.transpose(tf_matrix),sys_enl_op['ham']),tf_matrix),\
                       'sz':np.matmul(np.matmul(np.transpose(tf_matrix),sys_enl_op['sz']),tf_matrix),\
                       'sp':np.matmul(np.matmul(np.transpose(tf_matrix),sys_enl_op['sp']),tf_matrix),\
                       'sm':np.matmul(np.matmul(np.transpose(tf_matrix),sys_enl_op['sm']),tf_matrix),\
                       'id':np.matmul(np.matmul(np.transpose(tf_matrix),sys_enl_op['id']),tf_matrix)}
        new_block=Block(number_spins=sysblock_new.number_spins,basis_size=m,operator_dict=new_dictionary)
    return(new_block)

In [88]:
def finite_dmrg(N,m,sweeps):
    # LR: Left Right
    LR_operators={}
    BlockL=intial_block
    # sys: system block, and env: environment block
    LR_operators['sys',BlockL.number_spins]=BlockL
    LR_operators['env',BlockL.number_spins]=BlockL
    while BlockL.number_spins<N/2:
        BlockL=dmrg_step(BlockL,BlockL,m)
        LR_operators['sys',BlockL.number_spins]=BlockL
        LR_operators['env',BlockL.number_spins]=BlockL
    # Sweep part
    system_label='sys'
    env_label='env'
    system_block=BlockL
    del BlockL
    for i in range(1,sweeps+1):
        env_block=LR_operators[env_label,N-system_block.number_spins-1]
        system_block=dmrg_step(system_block,env_block,m)
        if env_block.number_spins == 2:
            # We've come to the end of the chain, so we reverse course.
            system_block, env_block = env_block, system_block
            system_label, env_label = env_label, system_label
        LR_operators[system_label, system_block.number_spins] = system_block

In [90]:
finite_dmrg(10,10,10)

[-3.3749326]
[-4.25802886]
[-4.25802837]
[-4.25802843]
[-4.25802843]
[-4.25802843]
[-4.25802843]
[-4.25802843]
[-4.25802848]
[-4.25802848]
